In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import seaborn as sns

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [61]:
df_train=pd.read_csv("train.csv")
df_test=pd.read_csv("test.csv")

---

In [62]:
cal = ["MSZoning","LotShape","HouseStyle","RoofStyle","MasVnrType","ExterQual","BsmtQual","BsmtExposure","KitchenQual","FireplaceQu","GarageType","GarageQual",]

In [63]:
x_train=df_train.loc[:,cal]
x_test=df_test.loc[:,cal]

In [64]:
x_train.loc[x_train.isnull()["GarageType"],:] = x_train.loc[x_train.isnull()["GarageType"],:].fillna(value="None")
x_train.loc[x_train.isnull()["GarageQual"],:] = x_train.loc[x_train.isnull()["GarageQual"],:].fillna(value="None")

In [65]:
x_train.loc[x_train.isnull()["FireplaceQu"],:] = x_train.loc[x_train.isnull()["FireplaceQu"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtQual"],:] = x_train.loc[x_train.isnull()["BsmtQual"],:].fillna(value="None")
x_train.loc[x_train.isnull()["BsmtExposure"],:] = x_train.loc[x_train.isnull()["BsmtExposure"],:].fillna(value="None")

In [67]:
x_train.loc[x_train.isnull()["MasVnrType"],:] = x_train.loc[x_train.isnull()["MasVnrType"],:].fillna(value="None")

In [68]:
x_test.loc[x_test.isnull()["GarageType"],:] = x_test.loc[x_test.isnull()["GarageType"],:].fillna(value="None")
x_test.loc[x_test.isnull()["GarageQual"],:] = x_test.loc[x_test.isnull()["GarageQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["FireplaceQu"],:] = x_test.loc[x_test.isnull()["FireplaceQu"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtQual"],:] = x_test.loc[x_test.isnull()["BsmtQual"],:].fillna(value="None")
x_test.loc[x_test.isnull()["BsmtExposure"],:] = x_test.loc[x_test.isnull()["BsmtExposure"],:].fillna(value="None")
x_test.loc[x_test.isnull()["MasVnrType"],:] = x_test.loc[x_test.isnull()["MasVnrType"],:].fillna(value="None")

In [69]:
x_train.shape

(1460, 12)

In [70]:
x_test.shape

(1459, 12)

In [71]:
add_x = pd.concat([x_train,x_test])

In [72]:
one_hot = pd.get_dummies(add_x)

In [73]:
train_onehot=one_hot.iloc[0:1460,:]

In [74]:
test_onehot=one_hot.iloc[1460:2920,:]

In [100]:
train_onehot.shape

(1460, 66)

----

In [46]:
from sklearn.preprocessing import MinMaxScaler 

In [101]:
x_data =df_train.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [102]:
x_data.isnull().sum().sort_values(ascending=False)

MasVnrArea      8
GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [103]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())

In [104]:
x_data = pd.concat([train_onehot,x_data], axis=1)

In [105]:
x_data = MinMaxScaler().fit_transform(x_data)

In [106]:
len(x_data)

1460

In [107]:
y_data = df_train.iloc[:,-1]

In [108]:
y_data = y_data.values
y_data=y_data.reshape(-1,1)

In [109]:
print(x_data.shape)
print(y_data.shape)

(1460, 78)
(1460, 1)


In [125]:
X = tf.placeholder(tf.float32, shape=[None,78])
Y = tf.placeholder(tf.float32, shape=[None,1])


W = tf.Variable(tf.random_normal([78,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
train = optimizer.minimize(cost)

In [126]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(1000001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 5000 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

0 Cost :  39039058000.0 
Preadiction
 [[-0.558702  ]
 [ 3.078043  ]
 [-0.7091    ]
 ...
 [ 1.7350874 ]
 [-1.3254751 ]
 [ 0.14733362]]
5000 Cost :  1087412900.0 
Preadiction
 [[202578.89]
 [200974.95]
 [223007.67]
 ...
 [237540.83]
 [140043.77]
 [131106.22]]
10000 Cost :  1055589760.0 
Preadiction
 [[201389.25]
 [198938.53]
 [218957.81]
 ...
 [237452.66]
 [139336.25]
 [132897.84]]
15000 Cost :  1044249100.0 
Preadiction
 [[200654.73]
 [197824.67]
 [217478.7 ]
 ...
 [237438.17]
 [139216.92]
 [133668.9 ]]
20000 Cost :  1038188800.0 
Preadiction
 [[200147.47]
 [197219.94]
 [216800.73]
 ...
 [237473.45]
 [139167.66]
 [134011.02]]
25000 Cost :  1034014300.0 
Preadiction
 [[199782.77]
 [196863.02]
 [216473.98]
 ...
 [237619.36]
 [139156.89]
 [134190.72]]
30000 Cost :  1030707140.0 
Preadiction
 [[199513.05]
 [196632.97]
 [216318.84]
 ...
 [237857.22]
 [139169.38]
 [134304.72]]
35000 Cost :  1027902600.0 
Preadiction
 [[199308.9 ]
 [196473.38]
 [216252.25]
 ...
 [238152.1 ]
 [139193.88]
 [1343

-----

In [127]:
x_data =df_test.loc[:,["OverallQual","YearBuilt","YearRemodAdd","MasVnrArea","TotalBsmtSF","1stFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","Fireplaces","GarageCars","GarageArea"]]

In [128]:
x_data.loc[x_data.isnull()["MasVnrArea"],:] = x_data.loc[x_data.isnull()["MasVnrArea"],:].fillna(value=x_data["MasVnrArea"].mean())
x_data.loc[x_data.isnull()["GarageCars"],:] = x_data.loc[x_data.isnull()["GarageCars"],:].fillna(value=x_data["GarageCars"].mean())
x_data.loc[x_data.isnull()["GarageArea"],:] = x_data.loc[x_data.isnull()["GarageArea"],:].fillna(value=x_data["GarageArea"].mean())
x_data.loc[x_data.isnull()["TotalBsmtSF"],:] = x_data.loc[x_data.isnull()["TotalBsmtSF"],:].fillna(value=x_data["TotalBsmtSF"].mean())

In [130]:
x_data.isnull().sum().sort_values(ascending=False)

GarageArea      0
GarageCars      0
Fireplaces      0
TotRmsAbvGrd    0
FullBath        0
GrLivArea       0
1stFlrSF        0
TotalBsmtSF     0
MasVnrArea      0
YearRemodAdd    0
YearBuilt       0
OverallQual     0
dtype: int64

In [131]:
x_data_test = pd.concat([test_onehot,x_data], axis=1)

In [132]:
x_data_test = MinMaxScaler().fit_transform(x_data_test)

In [134]:
x_data_test.shape

(1459, 78)

----

In [139]:
sess.run(hypothesis, feed_dict={X:x_data_test})

array([[107243.58],
       [156251.86],
       [168129.28],
       ...,
       [150560.83],
       [114047.44],
       [234207.62]], dtype=float32)

In [135]:
sample = pd.read_csv("sample_submission.csv")

In [137]:
sample['SalePrice']=sess.run(hypothesis, feed_dict={X:x_data_test})

In [140]:
sample.to_csv("sample4.csv",index=False)